Author: Prakrut Kansara (phk5e@virginia.edu)

Description: This file takes in MODIS ET hdf files and extracts subdataset corresponding to ET. It then mosaics all the different tiles into a single tile. It projects the input sinusoidal projection to WGS84 projection for output. It saves this output in GeoTIFF format. Finally, it converts 8-day ET data to monthly data (For the conversion, 8-day value is divided by 8 and then multiplied by number of days in corresponding month). The output files will be stored with the name: 'MODIS_ET_YYYYMM.tif'

Date created: 10th May, 2020

Date last updated: 14th May, 2020

Version: 1.0.0

Import the required modules

In [2]:
import os, gdal, osr, glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import rasterio

In [2]:
# Use this to find out the starting date and ending date 
print(pd.to_datetime('2019305', format= '%Y%j'))
print(pd.to_datetime('2019361', format= '%Y%j'))

2019-11-01 00:00:00
2019-12-27 00:00:00


_______________________________________________________________________________________________________________________________________________________________________
Inputs - Change all the inputs required for this code in the code block below. There is no need to change anything else in the code.

In [4]:
start_date = '2019-11-01'
end_date = '2019-12-27'

src_dir = 'C:/Users/prakr/Desktop/Sarah_et/data' # Give the full path to the folder/directory where data is stored
dst_dir = 'C:/Users/prakr/Desktop/Sarah_et/output' # Give the full path to the folder/directory where you want the output to be stored

subdataset_id = [0] # index of layers to be extracted. For MODIS ET data: ET_500m, LE_500m, PET_500m, PLE_500m, ET_QC_500m
                    # https://lpdaac.usgs.gov/products/mod16a2v006/
                    # The number of layers can be acquired by GetSubDatasets()
band_n = 1

scale_factor = 0.1 #https://lpdaac.usgs.gov/products/mod16a2v006/

# Set the boundary coordinates of the map to subset
lat_roi_max = 16
lat_roi_min = 2
lon_roi_max = 50
lon_roi_min = 32

nodata = 32760

date_start = 11
date_end = 18

_______________________________________________________________________________________________________________________________________________________________________
Step 1: Create a function to extract subdatasets from HDF files

In [5]:
def hdf_subdataset_extraction(hdf_files, dst_dir, subdataset_id, band_n):

    # Open the dataset
    global band_ds
    hdf_ds = gdal.Open(hdf_files, gdal.GA_ReadOnly)
#hdf_ds = gdal.Open(t, gdal.GA_ReadOnly)
    # Loop read data of specified bands from subdataset_id
    size_1dim = gdal.Open(hdf_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly).ReadAsArray().astype(np.int16).shape
    band_array = np.empty([size_1dim[0], size_1dim[1], len(subdataset_id)], dtype=int)
    for idn in range(len(subdataset_id)):
        band_ds = gdal.Open(hdf_ds.GetSubDatasets()[subdataset_id[idn]][0], gdal.GA_ReadOnly)
        # Read into numpy array
        band_array[:, :, idn] = band_ds.ReadAsArray().astype(np.int16)

    # Build output path
    band_path = os.path.join(dst_dir, os.path.basename(os.path.splitext(hdf_files)[0]) + "-ctd" + ".tif")
    # Write raster
    out_ds = gdal.GetDriverByName('GTiff').Create(band_path, band_ds.RasterXSize, band_ds.RasterYSize, band_n, #Number of bands
                                  gdal.GDT_Int16, ['COMPRESS=LZW', 'TILED=YES'])
    out_ds.SetGeoTransform(band_ds.GetGeoTransform())
    out_ds.SetProjection(band_ds.GetProjection())

    # Loop write each band to Geotiff file
    for idb in range(len(subdataset_id)):
        out_ds.GetRasterBand(idb+1).WriteArray(band_array[:, :, idb])
        out_ds.GetRasterBand(idb+1).SetNoDataValue(0)
    out_ds = None  #close dataset to write to disc

_______________________________________________________________________________________________________________________________________________________________________
Step 2: Generate the list of dates for the whole dataset

In [6]:
start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
delta_date = end_date - start_date

date_seq = []
for i in range(delta_date.days + 1):
    date_str = start_date + datetime.timedelta(days=i)
    date_seq.append(str(date_str.timetuple().tm_year) + str(date_str.timetuple().tm_yday).zfill(3))

_______________________________________________________________________________________________________________________________________________________________________
Step 3: Extract the subdataset to separate HDF files

In [7]:

hdf_files = sorted(glob.glob('data/*.hdf'))##

for idt in range(len(hdf_files)):
    hdf_subdataset_extraction(hdf_files[idt], dst_dir, subdataset_id, band_n)
    print(hdf_files[idt]) # Print the file being processed

data\MOD16A2GF.A2019305.h21v07.006.2020025232447.hdf
data\MOD16A2GF.A2019305.h21v08.006.2020026004413.hdf
data\MOD16A2GF.A2019305.h22v07.006.2020026014017.hdf
data\MOD16A2GF.A2019305.h22v08.006.2020026024452.hdf
data\MOD16A2GF.A2019313.h21v07.006.2020026013750.hdf
data\MOD16A2GF.A2019313.h21v08.006.2020026031247.hdf
data\MOD16A2GF.A2019313.h22v07.006.2020026033906.hdf
data\MOD16A2GF.A2019313.h22v08.006.2020026043249.hdf
data\MOD16A2GF.A2019321.h21v07.006.2020026035044.hdf
data\MOD16A2GF.A2019321.h21v08.006.2020026050447.hdf
data\MOD16A2GF.A2019321.h22v07.006.2020026052859.hdf
data\MOD16A2GF.A2019321.h22v08.006.2020026061454.hdf
data\MOD16A2GF.A2019329.h21v07.006.2020026053121.hdf
data\MOD16A2GF.A2019329.h21v08.006.2020026064013.hdf
data\MOD16A2GF.A2019329.h22v07.006.2020026072904.hdf
data\MOD16A2GF.A2019329.h22v08.006.2020026074835.hdf
data\MOD16A2GF.A2019337.h21v07.006.2020026073212.hdf
data\MOD16A2GF.A2019337.h21v08.006.2020026081522.hdf
data\MOD16A2GF.A2019337.h22v07.006.20200260935

_______________________________________________________________________________________________________________________________________________________________________
Step 4: Create virtual files to mosaic all the tiles into a single tile of raster and save it as GeoTIFF. Print the file names corresponding to mosaic output files

In [8]:
os.chdir(dst_dir)
tif_files = sorted(glob.glob('*.tif'))##
tif_files_group = []
for idt in range(len(date_seq)):
    tif_files_group_1day = [tif_files.index(i) for i in tif_files if 'A' + date_seq[idt] in i]
    tif_files_group.append(tif_files_group_1day)

vrt_options = gdal.BuildVRTOptions(resampleAlg='near', addAlpha=None, bandList=None)
for idt in range(len(tif_files_group)):
    if len(tif_files_group[idt]) != 0:
        tif_files_toBuild = [tif_files[i] for i in tif_files_group[idt]]
        vrt_files_name = '_'.join(tif_files[tif_files_group[idt][0]].split('.')[0:2])
        gdal.BuildVRT('mosaic_sinu_' + vrt_files_name + '.vrt', tif_files_toBuild, options=vrt_options)
        exec('mosaic_sinu_' + vrt_files_name + '= None')

dst_srs = osr.SpatialReference()
dst_srs.ImportFromEPSG(4326) # WGS 84 projection
dst_wkt = dst_srs.ExportToWkt()
error_threshold = 0.1  # error threshold
resampling = gdal.GRA_NearestNeighbour

vrt_files = sorted(glob.glob('*.vrt'))
for idt in range(len(vrt_files)):
    # Open file
    src_ds = gdal.Open(vrt_files[idt])
    mos_file_name = '_'.join(os.path.splitext(vrt_files[idt])[0].split('_')[2:4])
    # Call AutoCreateWarpedVRT() to fetch default values for target raster dimensions and geotransform
    tmp_ds = gdal.AutoCreateWarpedVRT(src_ds, None, dst_wkt, resampling, error_threshold)
    # Crop to CONUS extent and create the final warped raster
    dst_ds = gdal.Translate(mos_file_name + '.tif', tmp_ds,
                            projWin=[lon_roi_min, lat_roi_max, lon_roi_max, lat_roi_min])
    dst_ds = None

    print(mos_file_name)

MOD16A2GF_A2019305
MOD16A2GF_A2019313
MOD16A2GF_A2019321
MOD16A2GF_A2019329
MOD16A2GF_A2019337
MOD16A2GF_A2019345
MOD16A2GF_A2019353
MOD16A2GF_A2019361


_______________________________________________________________________________________________________________________________________________________________________
Step 5: Seperate the individual files corresponding to each month 

In [9]:
jan_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 1]
feb_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 2]
mar_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 3]
apr_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 4]
may_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 5]
jun_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 6]
jul_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 7]
aug_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 8]
sep_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 9]
oct_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 10]
nov_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 11]
dec_files = [name for name in os.listdir('.') if name.endswith('.tif') and 'ctd' not in name and pd.to_datetime(name[date_start:date_end],format='%Y%j').month == 12]

_______________________________________________________________________________________________________________________________________________________________________
Step 6: Create function to convert 8-day to monthly value

In [10]:
def non_empty_month():
    if len(jan_files) != 0:
        return jan_files
    if len(feb_files) != 0:
        return feb_files
    if len(mar_files) != 0:
        return mar_files
    if len(apr_files) != 0:
        return apr_files
    if len(may_files) != 0:
        return may_files
    if len(jun_files) != 0:
        return jun_files
    if len(jul_files) != 0:
        return jul_files
    if len(aug_files) != 0:
        return aug_files
    if len(sep_files) != 0:
        return sep_files
    if len(oct_files) != 0:
        return oct_files
    if len(nov_files) != 0:
        return nov_files
    if len(dec_files) != 0:
        return dec_files 

In [11]:
profile = rasterio.open(non_empty_month()[0]).profile
raster_shape = rasterio.open(non_empty_month()[0]).shape

def monthly(file_list):
    data = np.empty((raster_shape[0],raster_shape[1],1))
    if len(file_list) > 0:
        for i in range(len(file_list)):
            file_open = rasterio.open(file_list[i])
            file = file_open.read(1)
            file = np.reshape(file,(file.shape[0], file.shape[1], 1))
            data = np.append(data,file, axis=2)
            data_new = np.delete(data, 0, axis=2)
        data_new[data_new > nodata] = np.nan
        data_mean = np.nanmean(data_new, axis=2)
        data_final = (data_mean/8)*scale_factor*pd.to_datetime(file_list[0][date_start:date_end],format='%Y%j').days_in_month
        with rasterio.open('MODIS_ET_'+ str(pd.to_datetime(file_list[0][date_start:date_end],format='%Y%j').year) + str(pd.to_datetime(file_list[0][date_start:date_end],format='%Y%j').month).zfill(2) + '.tif', 'w', **profile) as dst:
                dst.write(data_final.astype(rasterio.int16), 1)

_______________________________________________________________________________________________________________________________________________________________________
Step 7: Use the function to create monthly files for the whole time series

In [12]:
monthly(jan_files)
monthly(feb_files)
monthly(mar_files)
monthly(apr_files)
monthly(may_files)
monthly(jun_files)
monthly(jul_files)
monthly(aug_files)
monthly(sep_files)
monthly(oct_files)
monthly(nov_files)
monthly(dec_files)